In [ ]:
# Selenium, allows you to control a browser programmatically. Can be used to scrape data.
# Already installed Selenium in virtual environment via terminal. Only imports neccessary here.
%pip install pandas as pd
%pip install --upgrade six
%pip install selenium==3.141.0
#%pip install beautifulsoup4
#%pip install requests

from selenium import webdriver
#from selenium.webdriver.chrome.service import Service
#from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
#from selenium.common.exceptions import NoSuchElementException
#from bs4 import BeautifulSoup
#import requests
import time
import pandas as pd
import re
import numpy as np

# Path for the driver saved on my windows computer
driver_path = r"C:\Windows\chromedriver\chromedriver.exe"

# Unlimited amount of rows displayed
pd.set_option('display.max_rows', None)

In [8]:
driver = webdriver.Chrome(executable_path=driver_path)
# Navigate to the website
driver.get("https://www.scrapethissite.com/")

# Perform Actions to get to the required data or text.

button = driver.find_element_by_css_selector("a.btn.btn-lg.btn-default")
#potential simplier syntax for later: button1 = driver.find_element_by_css_selector("button.Explore Sandbox")
button.click() 
time.sleep(1) # Wait for the content to load before moving on in this code.  

link = driver.find_element_by_css_selector("h3.page-title a[href='/pages/simple/']")
link.click()
time.sleep(1)  

# Create an empty DataFrame with the columns you want
columns = ['Country Name', 'Capital', 'Population', 'Area (km\u00b2)']
data = pd.DataFrame(columns=columns)

# Find all country elements
all_countries = driver.find_elements_by_css_selector("div.col-md-4.country")

# Loop through each country element and scrape the data
for country in all_countries:
    country_name = country.find_element_by_css_selector("h3.country-name").text
    country_capital = country.find_element_by_css_selector("span.country-capital").text
    country_population = int(country.find_element_by_css_selector("span.country-population").text)
    area_text = country.find_element_by_css_selector("span.country-area").text
    
    # Use regular expressions to check if the area value is in scientific notation
    if re.match(r'^\d+(?:\.\d+)?E\d+$', area_text):
        # Convert scientific notation to a float using float()
        country_area = float(area_text)
    else:
        # Convert decimal notation to a float using float()
        country_area = float(area_text)
    
# Create a temporary DataFrame for the current country
    temp_df = pd.DataFrame({
        'Country Name': [country_name],
        'Capital': [country_capital],
        'Population': [country_population],
        'Area (km\u00b2)': [country_area]
    })

    # Concatenate the temporary DataFrame with the main DataFrame
    data = pd.concat([data, temp_df], ignore_index=True)
# Close the browser
driver.quit()
data['Area (km\u00b2)'] = data['Area (km\u00b2)'].replace([0, 0.0], np.nan)
data['Persons per area'] = data['Population'] / data['Area (km\u00b2)']
data = data.set_index('Country Name')
# Print the DataFrame
print(data)

                                                          Capital  Population  \
Country Name                                                                    
Andorra                                          Andorra la Vella       84000   
United Arab Emirates                                    Abu Dhabi     4975593   
Afghanistan                                                 Kabul    29121286   
Antigua and Barbuda                                    St. John's       86754   
Anguilla                                               The Valley       13254   
Albania                                                    Tirana     2986952   
Armenia                                                   Yerevan     2968000   
Angola                                                     Luanda    13068161   
Antarctica                                                   None           0   
Argentina                                            Buenos Aires    41343201   
American Samoa              

In [9]:
tot_pop = data['Population'].sum()
avg_pop = data['Population'].mean()
min_pop = data['Population'].min()
max_pop = data['Population'].max()

tot_area = data['Area (km\u00b2)'].sum()
avg_area = data['Area (km\u00b2)'].mean()
min_area = data['Area (km\u00b2)'].min()
max_area = data['Area (km\u00b2)'].max()

avg_pop_per_km2 = data['Persons per area'].mean()
min_pop_per_km2 = data['Persons per area'].min()
max_pop_per_km2 = data['Persons per area'].max()

# Calculate the non-zero minimum value for 'Persons per area'
min_pop_per_km2 = data.loc[data['Persons per area'] > 0, 'Persons per area'].min()

# Create a new DataFrame to store the aggregated values
summary_data = pd.DataFrame({
    'Population': [tot_pop, avg_pop, min_pop, max_pop],
    'Area (km\u00b2)': [tot_area, avg_area, min_area, max_area],
    'Persons per area': [np.nan, avg_pop_per_km2, min_pop_per_km2, max_pop_per_km2]
}, index=['Total', 'Average', 'Minimum', 'Maximum'])

# Displays output floats in non-scientific notation
pd.options.display.float_format = '{:,.2f}'.format

# Print the summary DataFrame
print(summary_data)

              Population     Area (km²)  Persons per area
Total   6,861,418,895.00 149,909,229.69               NaN
Average    27,445,675.58     602,045.10            306.79
Minimum             0.00           0.44              0.01
Maximum 1,330,044,000.00  17,100,000.00         16,905.13


In [10]:
driver = webdriver.Chrome(executable_path=driver_path)
driver.maximize_window()
# Get the url links for the top X number of Crypto's data you want to analyze.
def get_top_X_cryptocurrencies(driver):
    
    Top_X_Currencies = 5 # Input a number which will represent the top "X" cryptocurrencies by marketcap to analyze.
    num_links_ = Top_X_Currencies*4 # Adjustment needed to collect all the links neccessary.
    start_ = 1
    step_ = 4 # Step to ignore all but every 4th link that was gathered.
    wait_time = 20 # Maximum length you want driver to wait until timeout error.
    wait = WebDriverWait(driver, wait_time)
    url = 'https://coinmarketcap.com'
    driver.get(url)
    top_links = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'tbody tr td a[href^="/currencies/"]')))[start_:num_links_:step_]
    return [link.get_attribute('href') for link in top_links] # Gets the href (link) for each coin.

top_links = get_top_X_cryptocurrencies(driver)
driver.quit()
top_links

['https://coinmarketcap.com/currencies/bitcoin/markets/',
 'https://coinmarketcap.com/currencies/ethereum/markets/',
 'https://coinmarketcap.com/currencies/tether/markets/',
 'https://coinmarketcap.com/currencies/bnb/markets/',
 'https://coinmarketcap.com/currencies/usd-coin/markets/']

In [11]:
driver = webdriver.Chrome(executable_path=driver_path)
driver.maximize_window()
# Gathers the required data from the markets data table at the top_links.
def get_top_market_data_for_each_coin(driver, link):
    max_retries = 3
    for _ in range(max_retries):
        try:
            wait_time = 5
            wait = WebDriverWait(driver, wait_time)
            
            driver.get(link)  # goes to the market page for the current coin

            time.sleep(2)  # Sleep for 2 seconds to let the page load

            table_location = (By.CSS_SELECTOR, 'table.cmc-table')
            first_row_location = (By.CSS_SELECTOR, 'table.cmc-table tbody tr:nth-of-type(1)')

            # Verify the table with all its elements are present and loaded.
            wait.until(EC.presence_of_all_elements_located(table_location))

            # Scroll to the first row of the table
            first_row_element = driver.find_element(*first_row_location)

            # Verify the visibility of the first row of the table
            wait.until(EC.visibility_of_element_located(first_row_location))

            td_elements = first_row_element.find_elements_by_tag_name("td")
            td_elements = [td for td in td_elements if td.get_attribute("style") != "display: none;"]  # Filter out hidden elements

            exchange = td_elements[1].text.strip()
            pair = td_elements[2].text.strip()
            price = td_elements[3].text.strip()
            plus_2_depth = td_elements[4].text.strip()
            minus_2_depth = td_elements[5].text.strip()
            volume = td_elements[6].text.strip()
            volume_pct = td_elements[7].text.strip()

            return {
                'Exchange': exchange,
                'Pair': pair,
                'Price': price,
                'plus_2_depth': plus_2_depth,
                'minus_2_depth': minus_2_depth,
                'Volume': volume,
                'Volume_Pct': volume_pct,
            }
        except (TimeoutException, IndexError) as e:
            print(f"Error: {e}. Retrying...")
            time.sleep(2)  # Sleep for 2 seconds before retrying
        else:
            break
    else:
        print(f"Failed to fetch data for {link} after {max_retries} retries.")
        return None

data = []

for link in top_links:
    top_market_data = get_top_market_data_for_each_coin(driver, link)
    if top_market_data is not None:
        data.append(top_market_data)
    else:
        print(f"Skipping data for {link}")

print(data)
driver.quit()

[{'Exchange': 'Binance', 'Pair': 'BTC/USDT', 'Price': '$30,224.38', 'plus_2_depth': '$32,387,728', 'minus_2_depth': '$31,089,065', 'Volume': '$1,809,124,394', 'Volume_Pct': '9.46%'}, {'Exchange': 'Binance', 'Pair': 'ETH/USDT', 'Price': '$2,093.79', 'plus_2_depth': '$14,329,616', 'minus_2_depth': '$15,168,126', 'Volume': '$822,158,808', 'Volume_Pct': '9.35%'}, {'Exchange': 'Binance', 'Pair': 'BTC/USDT', 'Price': '$1.0003', 'plus_2_depth': '$32,376,104', 'minus_2_depth': '$31,077,907', 'Volume': '$1,810,063,573', 'Volume_Pct': '5.71%'}, {'Exchange': 'Binance', 'Pair': 'BNB/USDT', 'Price': '$341.41', 'plus_2_depth': '$2,280,991', 'minus_2_depth': '$3,147,557', 'Volume': '$126,171,907', 'Volume_Pct': '17.51%'}, {'Exchange': 'Binance', 'Pair': 'USDC/USDT', 'Price': '$0.9999', 'plus_2_depth': '$22,193,132', 'minus_2_depth': '$26,107,996', 'Volume': '$391,896,974', 'Volume_Pct': '9.03%'}]


In [12]:
data = [{'Exchange': 'Binance', 'Pair': 'BTC/USDT', 'Price': '$30,158.16', 'plus_2_depth': '$28,973,175', 'minus_2_depth': '$36,154,025', 'Volume': '$1,802,972,328', 'Volume_Pct': '9.49%'},
        {'Exchange': 'Binance', 'Pair': 'ETH/USDT', 'Price': '$2,090.49', 'plus_2_depth': '$14,637,301', 'minus_2_depth': '$16,508,327', 'Volume': '$836,159,436', 'Volume_Pct': '9.46%'},
        {'Exchange': 'Binance', 'Pair': 'BTC/USDT', 'Price': '$1.0003', 'plus_2_depth': '$28,964,126', 'minus_2_depth': '$36,142,733', 'Volume': '$1,805,463,362', 'Volume_Pct': '5.70%'},
        {'Exchange': 'Binance', 'Pair': 'BNB/USDT', 'Price': '$341.99', 'plus_2_depth': '$2,211,700', 'minus_2_depth': '$3,116,035', 'Volume': '$130,113,180', 'Volume_Pct': '17.89%'},
        {'Exchange': 'Binance', 'Pair': 'USDC/USDT', 'Price': '$0.9999', 'plus_2_depth': '$18,450,719', 'minus_2_depth': '$26,550,371', 'Volume': '$399,951,075', 'Volume_Pct': '9.24%'}]

# Create a DataFrame using the data
df = pd.DataFrame(data)

# Set the 'Pair' column as the index
df.set_index('Pair', inplace=True)

print(df) # The second BTC/USDT pair is actually USDT, since that is the top pair for UDST.

          Exchange       Price plus_2_depth minus_2_depth          Volume  \
Pair                                                                        
BTC/USDT   Binance  $30,158.16  $28,973,175   $36,154,025  $1,802,972,328   
ETH/USDT   Binance   $2,090.49  $14,637,301   $16,508,327    $836,159,436   
BTC/USDT   Binance     $1.0003  $28,964,126   $36,142,733  $1,805,463,362   
BNB/USDT   Binance     $341.99   $2,211,700    $3,116,035    $130,113,180   
USDC/USDT  Binance     $0.9999  $18,450,719   $26,550,371    $399,951,075   

          Volume_Pct  
Pair                  
BTC/USDT       9.49%  
ETH/USDT       9.46%  
BTC/USDT       5.70%  
BNB/USDT      17.89%  
USDC/USDT      9.24%  
